# Tarea 6: Alineamiento múltimple de sequencias  

Alumnos : Marcos Muñoz y Juan Antonio Jiménez Guerrero  
Fecha:05/06/2018  
Curso: 3º Ingeniería de la Salud , PAB.  

##Objetivo: 
El objetivo de la tarea a desarrollar consiste en el uso de M2Align y elegir cualquier BaliBAE para obtener los ficheros VAR(El cual contiene las secuencias alineadas) y el fichero FUN (el cual contiene los valores a optimizar). Por lo tanto ek objetico de la práctica consiste en considerar los objetivos STRIKE, TC y SP y encontrar los alineamientos asociados a sus medianas y máximos valores.
Para ello hemos seleccionado el BaliBAE "BB11001" y con ello hemos obtenido los ficheros "FUN.BB11001.tsv" y "VAR.BB11001.


##Código
Para nuestro fin hemos implementado una clase ReadFunVar la cual se encargará de realizar la tarea que se pide.
Inicialmente el constructor de la clase se enarga de llamar al método read_fun el cual se encargará de leer el fichero fun y tras recorrerlo almacenar los valores que me piden junto a la línea en la que se encuentra.
Posteriormente se llamará al método save_fun_values() el cual recorrerá de nuevo el fichero para crear una serie de ficheros en los que se guardaran el mejor STRIKE , TC ,SP  junto al resto de valores que lo acompañan y lo mismo haremos con la mediana de cada valor.
A continuación se llamará a save_fun_values() el cual se encargará de escribir en los ficheros indicados las secuencias alineadas que corresponden a cada valor de STRIKE, TC y SP.

In [1]:
class ReadFunVar:
    def __init__(self, file_fun, file_var):
        '''
        Initialize class variables
        :param file_fun:
        :param file_var:
        '''
        self.best_sp = [0, 0]
        self.best_tc = [0, 0]
        self.best_strike = [0, 0]
        self.median_sp = [0, 0]
        self.median_tc = [0, 0]
        self.median_strike = [0, 0]
        self.file_fun = file_fun
        self.file_var = file_var
        self.read_fun()

    def read_fun(self):
        """
        It completes the initial parameters where [score, line_of_the_score]
        :return:
        """
        strikes_for_median = []
        tc_for_median = []
        sp_for_median = []
        try:
            file = open(self.file_fun, 'r')
            # Changed var line counter for enumerate
            for line_number, line in enumerate(file):
                data = line.split('\t')
                strike = float(data[0])
                tc = float(data[1])
                sp = float(data[2])

                strikes_for_median.append(strike)
                tc_for_median.append(tc)
                sp_for_median.append(sp)

                if self.best_strike[0] < strike:
                    self.best_strike[0] = strike
                    self.best_strike[1] = line_number+1

                if self.best_tc[0] < tc:
                    self.best_tc[0] = tc
                    self.best_tc[1] = line_number+1

                if self.best_sp[0] < sp:
                    self.best_sp[0] = sp
                    self.best_sp[1] = line_number+1
        except OSError as err:
            print("OS error: {0}".format(err))
        except ValueError:
            print("Could not convert data to an integer or data structure mismatch")

        position_of_the_median = int(len(strikes_for_median) / 2)

        sorted_strike = sorted(strikes_for_median)
        sorted_tc = sorted(tc_for_median)
        sorted_sp = sorted(sp_for_median)

        median_strike = sorted_strike[position_of_the_median]
        median_tc = sorted_tc[position_of_the_median]
        median_sp = sorted_sp[position_of_the_median]

        line_median_strike = strikes_for_median.index(median_strike) + 1
        line_median_TC = tc_for_median.index(median_tc) + 1
        line_median_SP = sp_for_median.index(median_sp) + 1

        self.median_sp = [median_sp, line_median_SP]
        self.median_tc = [median_tc, line_median_TC]
        self.median_strike = [median_strike, line_median_strike]

        file.close()

    def save_fun_value(self, filename_out, pair):
        '''
        Saves the line of the passed pair to the specified file.
        :param filename_out:
        :param pair:
        :return:
        '''
        global line
        with open(self.file_fun, 'r') as file_in:
            line = file_in.readline()
            i = 0
            while i < int(pair[1]):
                line = file_in.readline()
                i += 1
        with open(filename_out, 'w') as file_out:
            file_out.write(line)

    def save_var_value(self, filename_out_pair, pair):
        """
        Writes the sequences in a file
        """
        cnt_of_blocks = 1
        cnt_of_empty_lines = 0

        try:

            file = open(self.file_var, 'r')
            file_out = open(filename_out_pair, 'w')

            for line in file:

                if line[0] == '\n':
                    cnt_of_empty_lines = cnt_of_empty_lines + 1

                if cnt_of_blocks == pair[1] and line[0] != '\n' and cnt_of_empty_lines >= 0:
                    file_out.write(line + '\n')

                if cnt_of_empty_lines == 2:
                    cnt_of_blocks = cnt_of_blocks + 1
                    cnt_of_empty_lines = 0

            file.close()
            file_out.close()

        except OSError as err:
            print("OS error: {0}".format(err))

    def save_fun_values(self):
        '''
        Saves best fun values
        :return:
        '''
        self.save_fun_value('Best_strike_value', self.best_strike)
        self.save_fun_value('Best_tc_value', self.best_tc)
        self.save_fun_value('Best_sp_value', self.best_sp)

        self.save_fun_value('Median_strike_value', self.median_strike)
        self.save_fun_value('Median_tc_value', self.median_tc)
        self.save_fun_value('Median_sp_value', self.median_sp)

    def save_var_values(self):
        '''
        Saves best var values
        :return:
        '''
        self.save_var_value('Best_strike_seq', self.best_strike)
        self.save_var_value('Best_tc_seq', self.best_tc)
        self.save_var_value('Best_sp_seq', self.best_sp)

        self.save_var_value('Median_strike_seq', self.median_strike)
        self.save_var_value('Median_tc_seq', self.median_tc)
        self.save_var_value('Median_sp_seq', self.median_sp)


# Create new ReadFunVar object and get best values
read = ReadFunVar("FUN.BB11001.tsv", "VAR.BB11001.tsv")
# Save values to file
read.save_fun_values()
read.save_var_values()

A coninuación se muestra la clase creada para la realización de los tests unitarios. En dicha clase se comprueba que el método save_fun_value() funciona corréctamente así como el método save_var_value().
    

##Tarea opcional

Por último y como tarea final, se propone la creación de un componente Python para visualizar los alineamientos. El código implementado se encarga de leer el fichero que le pasemos y por cada hit, es decir, cuando dos o más secuencias estén alineadas correctamente ycoinciden las letras en una posición determinada, se pintará esa franja del mismo color, po lo que si no están alineadas justo en esa posición presentarán colores diferentes.